In [ ]:
import requests
import json
import pandas as pd
import geopandas as gpd
import re
from datetime import datetime, timedelta
from shapely import wkt

# URL base da API
BASE_URL = "https://vigent.saude.sp.gov.br/sisaweb_api/"

def sanitize_filename(nome):
    """
    Sanitiza o nome do arquivo, removendo ou substituindo caracteres especiais.
    """
    return re.sub(r'[^a-zA-Z0-9_\-]', '_', nome)

def get_municipios(nome=""):
    """
    Obtém a lista de municípios correspondentes ao nome fornecido.
    """
    try:
        response = requests.get(f"{BASE_URL}lista.php?nome={nome}")
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Erro ao obter municípios: {e}")
        return None

def get_dados(tipo, municipio_id, inicio, final):
    """
    Obtém dados específicos para um município, tipo e intervalo de datas.
    """
    try:
        url = f"{BASE_URL}dados.php?tipo={tipo}&id={municipio_id}&inicio={inicio}&final={final}&censitario=1"
        response = requests.get(url)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Erro ao obter dados para o município ID {municipio_id}: {e}")
        return None

def calcular_intervalo_semanas(semanas=3):
    """
    Calcula a data de início e de fim para o intervalo de semanas fornecido.
    """
    hoje = datetime.today()
    inicio = hoje - timedelta(weeks=semanas)
    return inicio.strftime('%Y-%m-%d'), hoje.strftime('%Y-%m-%d')

def extrair_dados_municipio(municipio_nome, semanas=3, tipos=[4]):
    """
    Extrai e retorna dados pertinentes para o município e intervalo de semanas especificados.
    """
    inicio, final = calcular_intervalo_semanas(semanas)
    municipios = get_municipios(municipio_nome)
    if not municipios:
        print(f"Erro ao recuperar municípios para o nome: {municipio_nome}.")
        return None

    # Encontra o município que corresponde exatamente ao nome fornecido
    municipio = next((m for m in municipios if m['nome'].lower() == municipio_nome.lower()), None)
    if not municipio:
        print(f"Município '{municipio_nome}' não encontrado.")
        return None

    municipio_id = municipio['id']
    nome_municipio = municipio['nome']

    dados_frames = []

    for tipo in tipos:
        dados = get_dados(tipo, municipio_id, inicio, final)
        if dados:
            df = pd.DataFrame(dados)
            dados_frames.append(df)

    if dados_frames:
        df_concatenado = pd.concat(dados_frames, ignore_index=True)
        print(f"Dados extraídos para o município: {nome_municipio}, para os últimos {semanas} semanas.")
        return df_concatenado
    else:
        print("Nenhum dado foi extraído.")
        return None

def ler_dados_filtrados(caminho_csv):
    """
    Lê o arquivo CSV filtrado e o converte em um GeoDataFrame, assumindo que a coluna 'geometry' está em WKT.
    """
    try:
        df = pd.read_csv(caminho_csv)
        if 'geometry' in df.columns:
            # Converte a coluna 'geometry' de WKT para objetos geométricos
            df['geometry'] = df['geometry'].apply(wkt.loads)
            gdf = gpd.GeoDataFrame(df, geometry='geometry')
            print("Dados filtrados carregados com sucesso como GeoDataFrame!")
            return gdf
        else:
            print("A coluna 'geometry' não está presente no CSV.")
            return None
    except Exception as e:
        print(f"Erro ao ler o arquivo CSV filtrado: {e}")
        return None

def realizar_merge(gdf_geo, df_dados, chave_geo='CD_SETOR', chave_dados='cd_setor'):
    """
    Realiza o merge entre o GeoDataFrame e o DataFrame de dados, mantendo todas as linhas do GeoDataFrame.
    """
    df_dados = df_dados.copy()
    if chave_dados in df_dados.columns:
        df_dados[chave_dados] = df_dados[chave_dados].astype(str)

    if chave_geo in gdf_geo.columns:
        gdf_geo[chave_geo] = gdf_geo[chave_geo].astype(str)

    gdf_merged = gdf_geo.merge(df_dados, how='left', left_on=chave_geo, right_on=chave_dados)
    print("Merge realizado com sucesso. Todas as linhas do GeoDataFrame foram mantidas.")
    return gdf_merged

def salvar_geodataframe_csv(gdf, caminho_csv):
    """
    Salva o GeoDataFrame em um arquivo CSV, convertendo a coluna 'geometry' para WKT.
    """
    try:
        gdf = gdf.copy()
        gdf['geometry'] = gdf['geometry'].apply(lambda geom: geom.wkt if geom else None)
        gdf.to_csv(caminho_csv, index=False)
        print(f"GeoDataFrame salvo com sucesso em CSV: {caminho_csv}")
    except Exception as e:
        print(f"Erro ao salvar o GeoDataFrame em CSV: {e}")

# Configurações
municipio_desejado = "são josé do rio preto"
semanas_para_extrair = 3
tipos_de_dados = [4]
caminho_dados_filtrados = r"C:/Users/celso/Desktop/WIKI_DENGUE/lab-soft-wiki/encyclopedia/data_analytics/Sisaweb/dados_filtrados.csv"

# Fluxo principal
if __name__ == "__main__":
    # Extrai os dados do município
    df_dados = extrair_dados_municipio(municipio_desejado, semanas=semanas_para_extrair, tipos=tipos_de_dados)
    if df_dados is not None:
        # Lê o arquivo filtrado como GeoDataFrame
        gdf_filtrado = ler_dados_filtrados(caminho_dados_filtrados)
        if gdf_filtrado is not None:
            # Realiza o merge mantendo todas as linhas do GeoDataFrame filtrado
            gdf_merged = realizar_merge(gdf_filtrado, df_dados)
            # Salva o resultado em um arquivo CSV
            caminho_para_salvar = "dados_merged.csv"
            salvar_geodataframe_csv(gdf_merged, caminho_para_salvar)


Dados extraídos para o município: SAO JOSE DO RIO PRETO, para os últimos 3 semanas.
Dados filtrados carregados com sucesso como GeoDataFrame!
Merge realizado com sucesso. Todas as linhas do GeoDataFrame foram mantidas.
GeoDataFrame salvo com sucesso em: dados_merged_2.csv


c:\Users\celso\Desktop\WIKI_DENGUE\lab-soft-wiki\venv\lib\site-packages\pyogrio\geopandas.py:662: UserWarning: 'crs' was not provided.  The output dataset will not have projection information defined and may not be usable in other systems.
  write(
c:\Users\celso\Desktop\WIKI_DENGUE\lab-soft-wiki\venv\lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: The filename extension should be 'gpkg' instead of 'csv' to conform to the GPKG specification.
  ogr_write(
